In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, Input, Dense, Activation, Flatten,Dropout
from keras.models import Sequential
from keras.optimizers import Adam,Adadelta
import tensorflow as tf
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

import matplotlib.pyplot as plt
import imageio
import copy
import cv2
import albumentations
from skimage.transform import rescale, resize
import tqdm

Using TensorFlow backend.


In [2]:
ori = np.load('./data/pubfig/wm/origdata.npy')
data = np.load('./data/pubfig/wm/bkdata.npy')
true_lab = np.load('./data/pubfig/wm/truelab.npy')
tar_lab = np.load('./data/pubfig/wm/tarlab.npy')

In [3]:
def fastSAT(img):
    aug = albumentations.ShiftScaleRotate(scale_limit=0.16,shift_limit=0.16,rotate_limit=4,p=1,border_mode=0) #$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_OD(img):
    aug = albumentations.OpticalDistortion(p=1) #$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_GAMMA(img):
    aug = albumentations.RandomGamma(p=1,gamma_limit=(60,60)) #$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_MED(img):
    aug = albumentations.MedianBlur(p=1,blur_limit=(5,5)) #$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_DOWN(img):
    aug = albumentations.Downscale(p=1,scale_min=0.25,scale_max=0.25)#$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_MPN(img):
    aug = albumentations.MultiplicativeNoise(p=1,multiplier=1.53)#$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_GAMMA2(img):
    aug = albumentations.RandomGamma(p=1,gamma_limit=(260,260)) #$$$$$$200+ 
    augmented = aug(image=(img*255).astype(np.uint8))
    auged = augmented['image']/255
    return auged

def defend_RAND(img,scalimit=1.3):
    maxvalue = np.int(img.shape[0] * scalimit)
    rnd = np.random.randint(img.shape[0],maxvalue,(1,))[0]
    rescaled = resize(img,(rnd,rnd))
    h_rem = maxvalue - rnd
    w_rem = maxvalue - rnd
    pad_left = np.random.randint(0,w_rem,(1,))[0]
    pad_right = w_rem - pad_left
    pad_top = np.random.randint(0,h_rem,(1,))[0]
    pad_bottom = h_rem - pad_top
    padded = np.pad(rescaled,((pad_top,pad_bottom),(pad_left,pad_right),(0,0)),'constant',constant_values = 0)
    padded = resize(padded,(img.shape[0],img.shape[0]))
    return padded

In [4]:
def defending(imgset):
    outset = np.zeros_like(imgset)
    for i,img in enumerate(imgset):
        defimg = np.copy(img)

        defimg = defend_OD(defimg)
        defimg = defend_GAMMA(defimg)

        defimg = defend_MED(defimg)
        
        defimg = defend_MPN(defimg)
        defimg = defend_GAMMA2(defimg)

        defimg = defend_DOWN(defimg)        
        defimg = defend_MED(defimg)
        
        defimg = cv2.resize(defimg,(180,180))
        defimg = defend_MED(defimg)
        defimg = cv2.resize(defimg,(224,224))
        
        defimg = defend_RAND(defimg,scalimit=1.1)
        defimg = fastSAT(defimg)
        
        outset[i] = np.copy(defimg)
    return outset

In [5]:
from keras_vggface.vggface import VGGFace
def vggface_model():
    hidden_dim = 4096
    NUM_CLASSES = 83
    vgg_model = VGGFace(model='vgg16',weights=None,include_top=False, input_shape=(224, 224, 3))
    last_layer = vgg_model.get_layer('pool5').output
    x = Flatten(name='flatten')(last_layer)
    x = Dense(hidden_dim, activation='relu', name='fc6')(x)
    x = Dense(hidden_dim, activation='relu', name='fc7')(x)
    out = Dense(NUM_CLASSES, activation='softmax', name='fc8')(x)
    pubfig_vgg_model = Model(vgg_model.input, out)
    # compiling
    lr_optimizer=Adadelta(lr = 0.03)
    pubfig_vgg_model.compile(loss='categorical_crossentropy', optimizer=lr_optimizer, metrics=['accuracy'])
    
    return pubfig_vgg_model

In [6]:
model = vggface_model()
model.load_weights('./model/backdoor_face_wm.h5')

In [7]:
#Baseline
lr_optimizer=Adadelta(lr = 0.03)
model.compile(loss='categorical_crossentropy', optimizer=lr_optimizer, metrics=['accuracy'])
_,acc = model.evaluate(ori*255,np.squeeze(np.eye(83)[true_lab]),batch_size=10)
print('Baseline ACC over cleansamples',acc)

200/200 [==============================] - 3s 13ms/step
Baseline ACC over cleansamples 0.9599999934434891


In [8]:
#Baseline
_,acc = model.evaluate(data*255,np.squeeze(np.eye(83)[tar_lab]),batch_size=10)
print('Baseline ASR over patched data',acc)

200/200 [==============================] - 1s 4ms/step
Baseline ASR over patched data 1.0


In [9]:
#Conduct Intensive Preprocess
defdata = defending(data)
defori = defending(ori)

In [10]:
#Inference (I)
_,acc = model.evaluate(defori*255,np.squeeze(np.eye(83)[true_lab]),batch_size=10)
print('ACC over cleansamples after the intensive defense',acc)

200/200 [==============================] - 1s 4ms/step
ACC over cleansamples after the intensive defense 0.3700000047683716


In [11]:
#Inference (I)
_,acc = model.evaluate(defdata*255,np.squeeze(np.eye(83)[tar_lab]),batch_size=10)
print('ASR over patched data after the intensive defense',acc)

200/200 [==============================] - 1s 4ms/step
ASR over patched data after the intensive defense 0.36500000655651094


In [12]:
import h5py 
f = h5py.File('./data/clean_pubfig_face_dataset.h5','r')   #打开h5文件
for name in f.keys():
    print(name)

test_img
test_labels
train_img
train_labels
train_mean
val_img
val_labels


In [54]:
#count from 0 to 9 to acquire 10000 clean samples
count = 9
new_ori = (np.asarray(f['train_img'][count*1000:(count+1)*1000]))/255
new_true_lab = np.asarray(f['train_labels'][count*1000:(count+1)*1000])
hot_lab = np.eye(83)[new_true_lab]

In [55]:
#GYM Fine-tuning with intensive preprocessed clean data
for i in range(5):
    defnewori = defending(new_ori)
    model.fit(defnewori*255,hot_lab,epochs=1, batch_size=10)

Epoch 1/1
1000/1000 [==============================] - 17s 17ms/step - loss: 0.7182 - acc: 0.8020
Epoch 1/1
1000/1000 [==============================] - 17s 17ms/step - loss: 0.3728 - acc: 0.8930
Epoch 1/1
1000/1000 [==============================] - 17s 17ms/step - loss: 0.1852 - acc: 0.9470
Epoch 1/1
1000/1000 [==============================] - 17s 17ms/step - loss: 0.1363 - acc: 0.9590
Epoch 1/1
1000/1000 [==============================] - 17s 17ms/step - loss: 0.1466 - acc: 0.9590


In [56]:
#Fine-tuning + Inference (I)
_,acc = model.evaluate(defori*255,np.squeeze(np.eye(83)[true_lab]),batch_size=10)
print('ACC over cleansamples after the intensive defense',acc)

200/200 [==============================] - 1s 3ms/step
ACC over cleansamples after the intensive defense 0.80499999076128


In [57]:
#Fine-tuning + Inference (I)
_,acc = model.evaluate(defdata*255,np.squeeze(np.eye(83)[tar_lab]),batch_size=10)
print('ASR over patched data after the intensive defense',acc)

200/200 [==============================] - 1s 4ms/step
ASR over patched data after the intensive defense 0.015000000223517418


In [58]:
def defending2(imgset):
    outset = np.zeros_like(imgset)
    for i,img in enumerate(imgset):
        defimg = np.copy(img)
        

#         defimg = defend_OD(defimg)
#         defimg = defend_GAMMA(defimg)

        defimg = defend_MED(defimg)
        
#         defimg = defend_MPN(defimg)
#         defimg = defend_GAMMA2(defimg)

#         defimg = defend_DOWN(defimg)        
#         defimg = defend_MED(defimg)
        
        defimg = cv2.resize(defimg,(180,180))
        defimg = defend_MED(defimg)
        defimg = cv2.resize(defimg,(224,224))

#         defimg = defend_RAND(defimg,scalimit=1.1)
        defimg = fastSAT(defimg)
        
        outset[i] = np.copy(defimg)
    return outset

In [59]:
defdata_L = defending2(data)
defori_L = defending2(ori)

In [60]:
#Fine-tuning + Inference (L)
_,acc = model.evaluate(defori_L*255,np.squeeze(np.eye(83)[true_lab]),batch_size=10)
print('ACC over cleansamples after the lightweight defense',acc)

200/200 [==============================] - 1s 4ms/step
ACC over cleansamples after the lightweight defense 0.9149999916553497


In [62]:
#Fine-tuning + Inference (L)
_,acc = model.evaluate(defdata_L*255,np.squeeze(np.eye(83)[tar_lab]),batch_size=10)
print('ASR over patched data after the intensive defense',acc)

200/200 [==============================] - 1s 4ms/step
ASR over patched data after the intensive defense 0.010000000149011612


In [63]:
model.save_weights('./model/fine/backdoor_face_wm.h5')